**Inputs**
* ImageCollections naive probabilities
* ImageCollections projected hazard magnitudes calculated from NEX-GDDP
* ImageCollections alpha, beta for each magnitude range

**Outputs**
* Images: maximum a posterior (MAP) probabilities of each magnitude-year pair

In [1]:
import ee
ee.Authenticate()

Enter verification code: 4/1AX4XfWhjI6aZoIqClc6iXnkumL1ugRT5RJPu1TGN1FTEQp_qztw1ea5b52I

Successfully saved authorization token.


In [2]:
ee.Initialize()

In [3]:
import sys
sys.setrecursionlimit(2000)

In [4]:
WHOLE_GLOBE = ee.Geometry.Rectangle([-179.999, -90, 180, 90], 'EPSG:4326', False)
MODELS = ['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M']
hazards = ['ds', 'maxdryspell']
CURRENT_YEAR = 2030
FUTURE_YEAR = 2065
START_YEAR = 2020
END_YEAR = 2099

In [5]:
hazard_info = {
    'ds': {
        'num_bins': 100,
        'bin_width': 1
    },
    'maxdryspell': {
        'num_bins': 73,
        'bin_width': 5
    }
}

In [6]:
nex_mags = {}
for h in hazards:
    nex_mags[h] = ee.ImageCollection('users/tedwongwri/dataportal/magnitudes/' + h)

In [7]:
def parameter_prime(param, target_year, mag_low, mag_high, flip):
    cnp = naive_current.select('magnitude_{0}-{1}'.format(str(mag_low), str(mag_high)))
    fnp = naive_future.select('magnitude_{0}-{1}'.format(str(mag_low), str(mag_high)))
    if flip:
        return param.add((((fnp.multiply(-1).add(1)).subtract(cnp.multiply(-1).add(1))).divide(FUTURE_YEAR - CURRENT_YEAR)).multiply(target_year - CURRENT_YEAR)).multiply(0.1)
    else:
        return param.add(((fnp.subtract(cnp)).divide(FUTURE_YEAR - CURRENT_YEAR)).multiply(target_year - CURRENT_YEAR)).multiply(0.1)

In [ ]:
for hazardname in hazards:
    alphabeta = ee.ImageCollection('users/tedwongwri/dataportal/alphabeta/' + hazardname)
    naive_current = ee.Image('users/tedwongwri/dataportal/naive-probs/' + hazardname + '_' + str(CURRENT_YEAR))
    naive_future = ee.Image('users/tedwongwri/dataportal/naive-probs/' + hazardname + '_' + str(FUTURE_YEAR))

    for year in range(START_YEAR, END_YEAR + 1):
        year_probs = ee.Image()
        for bin_idx in range(hazard_info[hazardname]['num_bins']):
            mag_low = (bin_idx) * hazard_info[hazardname]['bin_width']
            mag_high = (bin_idx + 1) * hazard_info[hazardname]['bin_width']
            successes = ee.Image.constant(0)
            failures = ee.Image.constant(0)
            year_magnitude_images_list = ee.List([])
            ab = alphabeta.filterMetadata('magnitude_low', 'equals', mag_low).first()
            alpha = parameter_prime(ab.select('alpha'), year, mag_low, mag_high, False)
            beta = parameter_prime(ab.select('beta'), year, mag_low, mag_high, True)
            for model in MODELS:
                hazard_year_model = nex_mags[hazardname].filterMetadata('year', 'equals', year).filterMetadata('model', 'equals', model).first()
                succ = hazard_year_model.gte(mag_low).multiply(1).multiply(hazard_year_model.lt(mag_high).multiply(1))
                successes = successes.add(succ)
                failures = failures.add(succ.multiply(-1).add(1))
                #successes = successes.add(hazard_year_model.round().eq(mag_img).multiply(1))
                #failures = failures.add(hazard_year_model.round().eq(mag_img).multiply(-1).add(1))
                year_magnitude_images = (successes.add(alpha).subtract(1)).divide(successes.add(failures).add(beta).add(alpha).subtract(2)).max(0).min(1)
                year_magnitude_images_list = year_magnitude_images_list.add(year_magnitude_images)
            year_magnitude_meanprob = ee.ImageCollection(year_magnitude_images_list).mean().rename('magnitude_{0}-{1}'.format(str(mag_low), str(mag_high)))
            year_probs = year_probs.addBands(year_magnitude_meanprob)
        year_probs = year_probs.set('year', year)
        year_probs = year_probs.select(
            year_probs.bandNames().filter(
                ee.Filter.stringStartsWith('item', 'magnitude_')
            )
        )
        task = ee.batch.Export.image.toAsset(**{
            'image': year_probs,
            'description': 'MAPprob_{0}_{1}'.format(hazardname, str(year)),
            'assetId': 'users/tedwongwri/dataportal/MAPprobs/{0}/{1}'.format(hazardname, str(year)),
            'region': WHOLE_GLOBE,
            'crs': 'EPSG:4326',
            'dimensions': '1440x720'
        })
        task.start()